<a href="https://colab.research.google.com/github/Nikflix/Project/blob/main/F1_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [78]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jtrotman/formula-1-race-data")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'formula-1-race-data' dataset.
Path to dataset files: /kaggle/input/formula-1-race-data


In [79]:
import os

# The 'path' variable is from your previous cell
file_list = os.listdir(path)

print("Files in the dataset directory:")
print(file_list)

Files in the dataset directory:
['races.csv', 'constructor_results.csv', 'drivers.csv', 'constructors.csv', 'lap_times.csv', 'status.csv', 'driver_standings.csv', 'seasons.csv', 'pit_stops.csv', 'sprint_results.csv', 'constructor_standings.csv', 'results.csv', 'circuits.csv', 'qualifying.csv']


In [80]:
path

'/kaggle/input/formula-1-race-data'

In [81]:
import pandas as pd
import os

# This is the path from your first cell
path = '/kaggle/input/formula-1-race-data'

# Choose a file to read from the list you generated above
file_to_read = 'races.csv'

# Create the full path to the file
# os.path.join() is the best way to combine paths
full_file_path = os.path.join(path, file_to_read)

print(f"Reading file: {full_file_path}")

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(full_file_path)

# Display the first 5 rows to confirm it worked
print("\nFirst 5 rows of the dataset:")
df.head()

Reading file: /kaggle/input/formula-1-race-data/races.csv

First 5 rows of the dataset:


,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [82]:
df.columns

Index(['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url',
       'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
       'quali_date', 'quali_time', 'sprint_date', 'sprint_time'],
      dtype='object')

CSV files in the directory:
['circuits.csv', 'seasons.csv', 'status.csv', 'driver_standings.csv', 'sprint_results.csv', 'constructor_standings.csv', 'races.csv', 'results.csv', 'constructor_results.csv', 'pit_stops.csv', 'qualifying.csv', 'constructors.csv', 'drivers.csv', 'lap_times.csv']


In [83]:
print(file_list)

['races.csv', 'constructor_results.csv', 'drivers.csv', 'constructors.csv', 'lap_times.csv', 'status.csv', 'driver_standings.csv', 'seasons.csv', 'pit_stops.csv', 'sprint_results.csv', 'constructor_standings.csv', 'results.csv', 'circuits.csv', 'qualifying.csv']


In [84]:
dataframes = {}
for file_name in file_list:
    if file_name.endswith('.csv'):
        full_file_path = os.path.join(path, file_name)
        try:
            df_name = file_name.replace('.csv', '')
            dataframes[df_name] = pd.read_csv(full_file_path)
            print(f"Successfully read {file_name} into DataFrame '{df_name}'")
        except FileNotFoundError:
            print(f"Error: File not found at {full_file_path}")
        except Exception as e:
            print(f"Error reading {file_name}: {e}")


Successfully read races.csv into DataFrame 'races'
Successfully read constructor_results.csv into DataFrame 'constructor_results'
Successfully read drivers.csv into DataFrame 'drivers'
Successfully read constructors.csv into DataFrame 'constructors'
Successfully read lap_times.csv into DataFrame 'lap_times'
Successfully read status.csv into DataFrame 'status'
Successfully read driver_standings.csv into DataFrame 'driver_standings'
Successfully read seasons.csv into DataFrame 'seasons'
Successfully read pit_stops.csv into DataFrame 'pit_stops'
Successfully read sprint_results.csv into DataFrame 'sprint_results'
Successfully read constructor_standings.csv into DataFrame 'constructor_standings'
Successfully read results.csv into DataFrame 'results'
Successfully read circuits.csv into DataFrame 'circuits'
Successfully read qualifying.csv into DataFrame 'qualifying'


In [85]:
results_df = dataframes['results']
races_df = dataframes['races']
drivers_df = dataframes['drivers']
constructors_df = dataframes['constructors']

# Merge results and races
consolidated_df = pd.merge(results_df, races_df, on='raceId', how='left')

# Merge with drivers
consolidated_df = pd.merge(consolidated_df, drivers_df, on='driverId', how='left')

# Merge with constructors
consolidated_df = pd.merge(consolidated_df, constructors_df, on='constructorId', how='left')

# Display the first few rows to verify
display(consolidated_df.head())

,resultId,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,...,code,forename,surname,dob,nationality_x,url_y,constructorRef,name_y,nationality_y,url
0,1,18,1,1,22,1,1,1,1,10.0,...,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,18,2,2,3,5,2,2,2,8.0,...,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,18,3,3,7,7,3,3,3,6.0,...,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,18,4,4,5,11,4,4,4,5.0,...,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,18,5,1,23,3,5,5,5,4.0,...,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren


In [86]:
len(consolidated_df)

27078

In [87]:
consolidated_df.info()
print("\nMissing values per column:")
print(consolidated_df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27078 entries, 0 to 27077
Data columns (total 47 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         27078 non-null  int64  
 1   raceId           27078 non-null  int64  
 2   driverId         27078 non-null  int64  
 3   constructorId    27078 non-null  int64  
 4   number_x         27078 non-null  object 
 5   grid             27078 non-null  int64  
 6   position         27078 non-null  object 
 7   positionText     27078 non-null  object 
 8   positionOrder    27078 non-null  int64  
 9   points           27078 non-null  float64
 10  laps             27078 non-null  int64  
 11  time_x           27078 non-null  object 
 12  milliseconds     27078 non-null  object 
 13  fastestLap       27078 non-null  object 
 14  rank             27078 non-null  object 
 15  fastestLapTime   27078 non-null  object 
 16  fastestLapSpeed  27078 non-null  object 
 17  statusId    

In [88]:
consolidated_df.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time_x',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'round', 'circuitId', 'name_x',
       'date', 'time_y', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date',
       'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time',
       'sprint_date', 'sprint_time', 'driverRef', 'number_y', 'code',
       'forename', 'surname', 'dob', 'nationality_x', 'url_y',
       'constructorRef', 'name_y', 'nationality_y', 'url'],
      dtype='object')

In [12]:
consolidated_df.dropna(subset=['url_x', 'url_y', 'url','nationality_x', 'nationality_y', 'quali_date', 'sprint_date', 'fp1_date', 'statusId', 'fp1_date', 'fp2_date', 'fp3_date', 'quali_date', 'sprint_date'], inplace=True)

# Verify that missing values in these columns are handled
print("\nMissing values per column after handling:")
print(consolidated_df.isnull().sum())


Missing values per column after handling:
resultId           0
raceId             0
driverId           0
constructorId      0
number_x           0
grid               0
position           0
positionText       0
positionOrder      0
points             0
laps               0
time_x             0
milliseconds       0
fastestLap         0
rank               0
fastestLapTime     0
fastestLapSpeed    0
statusId           0
year               0
round              0
circuitId          0
name_x             0
date               0
time_y             0
url_x              0
fp1_date           0
fp1_time           0
fp2_date           0
fp2_time           0
fp3_date           0
fp3_time           0
quali_date         0
quali_time         0
sprint_date        0
sprint_time        0
driverRef          0
number_y           0
code               0
forename           0
surname            0
dob                0
nationality_x      0
url_y              0
constructorRef     0
name_y             0
nationality_

In [32]:
data_df = consolidated_df
data_df.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time_x',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'round', 'circuitId', 'name_x',
       'date', 'time_y', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date',
       'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time',
       'sprint_date', 'sprint_time', 'driverRef', 'number_y', 'code',
       'forename', 'surname', 'dob', 'nationality_x', 'url_y',
       'constructorRef', 'name_y', 'nationality_y', 'url'],
      dtype='object')

In [89]:
consolidated_df.drop(['url_x', 'url_y', 'url','nationality_x', 'nationality_y', 'quali_date', 'sprint_date', 'fp1_date', 'fp2_date', 'fp3_date', 'statusId'], axis=1, inplace=True)

In [90]:
consolidated_df.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time_x',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'year', 'round', 'circuitId', 'name_x', 'date',
       'time_y', 'fp1_time', 'fp2_time', 'fp3_time', 'quali_time',
       'sprint_time', 'driverRef', 'number_y', 'code', 'forename', 'surname',
       'dob', 'constructorRef', 'name_y'],
      dtype='object')

In [91]:
data_df = consolidated_df
data_df.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time_x',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'year', 'round', 'circuitId', 'name_x', 'date',
       'time_y', 'fp1_time', 'fp2_time', 'fp3_time', 'quali_time',
       'sprint_time', 'driverRef', 'number_y', 'code', 'forename', 'surname',
       'dob', 'constructorRef', 'name_y'],
      dtype='object')

In [92]:
data_df.head()

,resultId,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,...,quali_time,sprint_time,driverRef,number_y,code,forename,surname,dob,constructorRef,name_y
0,1,18,1,1,22,1,1,1,1,10.0,...,\N,\N,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,mclaren,McLaren
1,2,18,2,2,3,5,2,2,2,8.0,...,\N,\N,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,bmw_sauber,BMW Sauber
2,3,18,3,3,7,7,3,3,3,6.0,...,\N,\N,rosberg,6,ROS,Nico,Rosberg,1985-06-27,williams,Williams
3,4,18,4,4,5,11,4,4,4,5.0,...,\N,\N,alonso,14,ALO,Fernando,Alonso,1981-07-29,renault,Renault
4,5,18,5,1,23,3,5,5,5,4.0,...,\N,\N,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,mclaren,McLaren


In [29]:
data_df = df.drop(['url', 'url_y', 'fp1_date', 'fp2_date', 'fp3_date', 'quali_date', 'sprint_time', 'url_x', 'code', 'statusId'],
             axis=1, inplace = True, errors='ignore')

In [93]:
data_df.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time_x',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'year', 'round', 'circuitId', 'name_x', 'date',
       'time_y', 'fp1_time', 'fp2_time', 'fp3_time', 'quali_time',
       'sprint_time', 'driverRef', 'number_y', 'code', 'forename', 'surname',
       'dob', 'constructorRef', 'name_y'],
      dtype='object')

In [94]:
print("\nMissing values per column:")
print(data_df.isnull().sum())


Missing values per column:
resultId             0
raceId               0
driverId             0
constructorId        0
number_x             0
grid                 0
position             0
positionText         0
positionOrder        0
points               0
laps                 0
time_x               0
milliseconds         0
fastestLap           0
rank                 0
fastestLapTime       0
fastestLapSpeed      0
year                 0
round                0
circuitId            0
name_x               0
date                 0
time_y               0
fp1_time           319
fp2_time           319
fp3_time           319
quali_time         319
sprint_time        319
driverRef            0
number_y             0
code                 0
forename             0
surname              0
dob                  0
constructorRef       0
name_y               0
dtype: int64


# Task
Fill the null values in the columns `fp1_time`, `fp2_time`, `fp3_time`, and `quali_time` with the mean of their respective columns. Handle '\N' values by converting them to NaN before calculating the mean and filling.

## Define a conversion function

### Subtask:
Create a function to convert the time strings (including '\N') into a numerical representation (e.g., seconds). Handle the '\N' values by converting them to NaN.


**Reasoning**:
Define a function to convert time strings to seconds, handling '\N' and potential time formats.



In [95]:
def time_to_seconds(time_str):
    if time_str == '\\N':
        return np.nan
    try:
        # Handle 'MM:SS.sss' format
        if '.' in time_str:
            minutes, seconds = time_str.split(':')
            return int(minutes) * 60 + float(seconds)
        # Handle 'HH:MM:SS' format (less common in lap/session times but good to handle)
        elif time_str.count(':') == 2:
            hours, minutes, seconds = time_str.split(':')
            return int(hours) * 3600 + int(minutes) * 60 + int(seconds)
        else:
            return np.nan
    except:
        return np.nan


## Apply the conversion function

### Subtask:
Apply the conversion function to the specified time columns (`fp1_time`, `fp2_time`, `fp3_time`, `quali_time`) to convert them to numerical type.


**Reasoning**:
Apply the defined `time_to_seconds` function to the specified time columns in the `data_df` DataFrame to convert their values to seconds and handle '\N' by converting them to NaN.



In [96]:
time_columns_to_convert = ['fp1_time', 'fp2_time', 'fp3_time', 'quali_time', 'sprint_time']

for col in time_columns_to_convert:
    data_df[col] = data_df[col].apply(time_to_seconds)

display(data_df.head())

,resultId,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,...,quali_time,sprint_time,driverRef,number_y,code,forename,surname,dob,constructorRef,name_y
0,1,18,1,1,22,1,1,1,1,10.0,...,NaN,NaN,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,mclaren,McLaren
1,2,18,2,2,3,5,2,2,2,8.0,...,NaN,NaN,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,bmw_sauber,BMW Sauber
2,3,18,3,3,7,7,3,3,3,6.0,...,NaN,NaN,rosberg,6,ROS,Nico,Rosberg,1985-06-27,williams,Williams
3,4,18,4,4,5,11,4,4,4,5.0,...,NaN,NaN,alonso,14,ALO,Fernando,Alonso,1981-07-29,renault,Renault
4,5,18,5,1,23,3,5,5,5,4.0,...,NaN,NaN,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,mclaren,McLaren


## Calculate the mean

### Subtask:
Calculate the mean for each of the converted numerical time columns, ignoring NaN values.


**Reasoning**:
Calculate the mean for each of the converted numerical time columns, ignoring NaN values.



In [75]:
print(data_df.isnull().sum())

resultId               0
raceId                 0
driverId               0
constructorId          0
number_x               0
grid                   0
position               0
positionText           0
positionOrder          0
points                 0
laps                   0
time_x                 0
milliseconds           0
fastestLap             0
rank                   0
fastestLapTime         0
fastestLapSpeed        0
year                   0
round                  0
circuitId              0
name_x                 0
date                   0
time_y                 0
fp1_time           27078
fp2_time           27078
fp3_time           27078
quali_time         27078
sprint_time        26778
driverRef              0
number_y               0
code                   0
forename               0
surname                0
dob                    0
constructorRef         0
name_y                 0
dtype: int64


In [97]:
time_columns = ['fp1_time', 'fp2_time','sprint_time', 'fp3_time', 'quali_time']
mean_times = data_df[time_columns].mean()
print(mean_times)

fp1_time       41585.430464
fp2_time       51986.754967
sprint_time    53520.000000
fp3_time       38408.498584
quali_time     52278.145695
dtype: float64


## Impute missing values

### Subtask:
Fill the NaN values in each of the converted numerical time columns with the calculated mean for that column.


**Reasoning**:
Fill the NaN values in the specified time columns with the calculated mean for each column.



In [98]:
time_columns = ['fp1_time', 'fp2_time', 'fp3_time', 'quali_time', 'sprint_time']
data_df[time_columns] = data_df[time_columns].fillna(mean_times)
display(data_df.head())

,resultId,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,...,quali_time,sprint_time,driverRef,number_y,code,forename,surname,dob,constructorRef,name_y
0,1,18,1,1,22,1,1,1,1,10.0,...,52278.145695,53520.0,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,mclaren,McLaren
1,2,18,2,2,3,5,2,2,2,8.0,...,52278.145695,53520.0,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,bmw_sauber,BMW Sauber
2,3,18,3,3,7,7,3,3,3,6.0,...,52278.145695,53520.0,rosberg,6,ROS,Nico,Rosberg,1985-06-27,williams,Williams
3,4,18,4,4,5,11,4,4,4,5.0,...,52278.145695,53520.0,alonso,14,ALO,Fernando,Alonso,1981-07-29,renault,Renault
4,5,18,5,1,23,3,5,5,5,4.0,...,52278.145695,53520.0,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,mclaren,McLaren


**Reasoning**:
Verify that the NaN values have been filled in the specified columns.



In [99]:
print("\nMissing values after filling with mean:")
print(data_df[time_columns].isnull().sum())


Missing values after filling with mean:
fp1_time       0
fp2_time       0
fp3_time       0
quali_time     0
sprint_time    0
dtype: int64


In [100]:
print("\nMissing values per column:")
print(data_df.isnull().sum())


Missing values per column:
resultId           0
raceId             0
driverId           0
constructorId      0
number_x           0
grid               0
position           0
positionText       0
positionOrder      0
points             0
laps               0
time_x             0
milliseconds       0
fastestLap         0
rank               0
fastestLapTime     0
fastestLapSpeed    0
year               0
round              0
circuitId          0
name_x             0
date               0
time_y             0
fp1_time           0
fp2_time           0
fp3_time           0
quali_time         0
sprint_time        0
driverRef          0
number_y           0
code               0
forename           0
surname            0
dob                0
constructorRef     0
name_y             0
dtype: int64


## Verify changes

### Subtask:
Display the head of the DataFrame and check for null values in the processed columns to confirm the imputation.


**Reasoning**:
Display the head of the DataFrame and check for null values in the processed columns to confirm the imputation.



In [101]:
# Display the first few rows to visually inspect the imputed values
display(data_df.head())

# Check for remaining null values in the processed columns
print("\nMissing values after filling with mean:")
print(data_df[['fp1_time', 'fp2_time', 'fp3_time', 'quali_time']].isnull().sum())

,resultId,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,...,quali_time,sprint_time,driverRef,number_y,code,forename,surname,dob,constructorRef,name_y
0,1,18,1,1,22,1,1,1,1,10.0,...,52278.145695,53520.0,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,mclaren,McLaren
1,2,18,2,2,3,5,2,2,2,8.0,...,52278.145695,53520.0,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,bmw_sauber,BMW Sauber
2,3,18,3,3,7,7,3,3,3,6.0,...,52278.145695,53520.0,rosberg,6,ROS,Nico,Rosberg,1985-06-27,williams,Williams
3,4,18,4,4,5,11,4,4,4,5.0,...,52278.145695,53520.0,alonso,14,ALO,Fernando,Alonso,1981-07-29,renault,Renault
4,5,18,5,1,23,3,5,5,5,4.0,...,52278.145695,53520.0,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,mclaren,McLaren



Missing values after filling with mean:
fp1_time      0
fp2_time      0
fp3_time      0
quali_time    0
dtype: int64


In [102]:
data_df.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time_x',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'year', 'round', 'circuitId', 'name_x', 'date',
       'time_y', 'fp1_time', 'fp2_time', 'fp3_time', 'quali_time',
       'sprint_time', 'driverRef', 'number_y', 'code', 'forename', 'surname',
       'dob', 'constructorRef', 'name_y'],
      dtype='object')

In [112]:
data_df.head()

,resultId,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,...,quali_time,sprint_time,driverRef,number_y,code,forename,surname,dob,constructorRef,name_y
0,1,18,1,1,22,1,1,1,1,10.0,...,52278.145695,53520.0,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,mclaren,McLaren
1,2,18,2,2,3,5,2,2,2,8.0,...,52278.145695,53520.0,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,bmw_sauber,BMW Sauber
2,3,18,3,3,7,7,3,3,3,6.0,...,52278.145695,53520.0,rosberg,6,ROS,Nico,Rosberg,1985-06-27,williams,Williams
3,4,18,4,4,5,11,4,4,4,5.0,...,52278.145695,53520.0,alonso,14,ALO,Fernando,Alonso,1981-07-29,renault,Renault
4,5,18,5,1,23,3,5,5,5,4.0,...,52278.145695,53520.0,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,mclaren,McLaren


In [118]:
# Filter data_df for years between 2006 and 2025
data_df_filtered = data_df[(data_df['year'] >= 2006) & (data_df['year'] <= 2025)].copy()


# Prepare data for constructor championship prediction using only columns from data_df_filtered
# Relevant columns in data_df_filtered for constructor championship are likely 'year', 'raceId', 'constructorId', and 'points'.
# Group by year, raceId, and constructorId to get points per constructor per race.
constructor_championship_df = data_df_filtered.groupby(['year', 'raceId', 'constructorId'])['points'].sum().reset_index()

# Calculate the cumulative points for each constructor across the seasons
constructor_championship_df = constructor_championship_df.sort_values(by=['constructorId', 'year', 'raceId'])
constructor_championship_df['cumulative_points'] = constructor_championship_df.groupby('constructorId')['points'].cumsum()

# Identify the constructor with the highest cumulative points at the end of each season to determine the champion
last_race_per_year = constructor_championship_df.groupby('year')['raceId'].max().reset_index()
championship_data = constructor_championship_df.merge(last_race_per_year, on=['year', 'raceId'], how='inner')

# Determine the championship winner for each year based on cumulative points at the last race
championship_winners = championship_data.loc[championship_data.groupby('year')['cumulative_points'].idxmax()].copy()
championship_winners['won_championship'] = 1

# Merge the target variable ('won_championship') back into the championship_data DataFrame
# Drop 'won_championship' if it exists before merging to avoid duplicates
if 'won_championship' in championship_data.columns:
    championship_data = championship_data.drop(columns=['won_championship'])

championship_data = championship_data.merge(championship_winners[['year', 'constructorId', 'won_championship']],
                                              on=['year', 'constructorId'], how='left')
championship_data['won_championship'] = championship_data['won_championship'].fillna(0)

# Display the first few rows and information
print("Constructor Championship Data:")
display(championship_data.head())
championship_data.info()

Constructor Championship Data:


,year,raceId,constructorId,points,cumulative_points,won_championship
0,2006,70,1,5.0,110.0,0.0
1,2007,52,1,8.0,328.0,0.0
2,2008,35,1,6.0,479.0,0.0
3,2009,17,1,0.0,550.0,0.0
4,2010,355,1,33.0,1004.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               210 non-null    int64  
 1   raceId             210 non-null    int64  
 2   constructorId      210 non-null    int64  
 3   points             210 non-null    float64
 4   cumulative_points  210 non-null    float64
 5   won_championship   210 non-null    float64
dtypes: float64(3), int64(3)
memory usage: 10.0 KB


In [113]:
import pandas as pd
import numpy as np

# Create the constructor_performance_stats DataFrame
constructor_performance_stats = pd.DataFrame({
    'constructorId': [1, 2, 3, 4],
    'avg_points_per_race': [0.777778, 0.444444, 0.333333, 0.277778],
    'std_points_per_race': [0.0, 0.0, 0.0, 0.0]
})

# Display the DataFrame
print("Constructor Performance Stats:")
print(constructor_performance_stats)

# If you want to match the exact formatting from your image:
print("\nFormatted display:")
print(constructor_performance_stats.to_string(index=True))

Constructor Performance Stats:
   constructorId  avg_points_per_race  std_points_per_race
0              1             0.777778                  0.0
1              2             0.444444                  0.0
2              3             0.333333                  0.0
3              4             0.277778                  0.0

Formatted display:
   constructorId  avg_points_per_race  std_points_per_race
0              1             0.777778                  0.0
1              2             0.444444                  0.0
2              3             0.333333                  0.0
3              4             0.277778                  0.0


In [119]:
# Calculate historical average and standard deviation of points per race for each constructor
# Using data_df filtered for 2006 onwards
data_2006_onwards = data_df[data_df['year'] >= 2006].copy()

constructor_historical_stats = data_2006_onwards.groupby('constructorId')['points'].agg(['mean', 'std']).reset_index()
constructor_historical_stats.columns = ['constructorId', 'avg_points_per_race', 'std_points_per_race']

# Fill NaN std_points_per_race with 0 (for constructors with only one race)
constructor_historical_stats['std_points_per_race'] = constructor_historical_stats['std_points_per_race'].fillna(0)

# Merge with constructors_df to get constructor names
constructor_historical_stats = constructor_historical_stats.merge(constructors_df[['constructorId', 'name']], on='constructorId', how='left')

print("Constructor Historical Performance Stats (2006 onwards):")
display(constructor_historical_stats.head())


# --- Bayesian Monte Carlo Simulation ---
n_simulations = 5000 # Increased simulations for better stability
# Get the number of races in 2025 from races_df (assuming it's available and up-to-date)
# races_df should contain the 2025 race schedule
n_races_2025 = races_df[races_df['year'] == 2025].shape[0]

championship_wins = {}

# Iterate through each simulation
for _ in range(n_simulations):
    simulated_points = {}
    # Iterate through each constructor's historical stats
    for index, row in constructor_historical_stats.iterrows():
        constructor_id = row['constructorId']
        avg_points = row['avg_points_per_race']
        std_points = row['std_points_per_race']

        # Simulate points for each race in 2025 based on a normal distribution
        # Ensure simulated points are not negative
        # Add a small value to std_points if it's zero to avoid zero variance issues in simulation
        simulated_race_points = np.maximum(0, np.random.normal(avg_points, max(std_points, 0.01), n_races_2025))


        # Sum up simulated points for the season
        simulated_points[constructor_id] = simulated_race_points.sum()

    # Determine the winner for this simulation
    if simulated_points: # Check if simulated_points is not empty
        # Find the constructor(s) with the maximum simulated points
        max_points = max(simulated_points.values())
        # Handle ties by considering all constructors with max points as winners in this simulation
        simulated_championship_winners = [
            constructor_id for constructor_id, points in simulated_points.items() if points == max_points
        ]

        # Increment win count for each winner in this simulation (handles ties)
        for winner_id in simulated_championship_winners:
             championship_wins[winner_id] = championship_wins.get(winner_id, 0) + (1 / len(simulated_championship_winners)) # Fractional win for ties


# Calculate winning percentage for each constructor
total_simulations = n_simulations # Use total simulations as the denominator
constructor_win_percentage = {
    constructor_id: (wins / total_simulations) * 100
    for constructor_id, wins in championship_wins.items()
}

# Convert to a DataFrame for display
constructor_win_percentage_df = pd.DataFrame(list(constructor_win_percentage.items()), columns=['constructorId', 'win_percentage'])

# Merge with constructors_df to get constructor names (assuming constructors_df is available)
constructor_win_percentage_df = constructor_win_percentage_df.merge(constructors_df[['constructorId', 'name']], on='constructorId', how='left')

# Sort by win percentage in descending order
constructor_win_percentage_df = constructor_win_percentage_df.sort_values(by='win_percentage', ascending=False)

print("\nSimulated Constructor Championship Winning Chances for 2025 (Bayesian Monte Carlo):")
display(constructor_win_percentage_df[['name', 'win_percentage']].round(2))

Constructor Historical Performance Stats (2006 onwards):


,constructorId,avg_points_per_race,std_points_per_race,name
0,1,5.887468,6.986552,McLaren
1,2,2.200000,2.587185,BMW Sauber
2,3,1.565301,3.701813,Williams
3,4,2.543269,3.766441,Renault
4,5,0.932836,2.185587,Toro Rosso



Simulated Constructor Championship Winning Chances for 2025 (Bayesian Monte Carlo):


,name,win_percentage
1,Mercedes,77.38
0,Red Bull,19.06
2,Ferrari,3.56


# Task
Use data_df as the main dataframe and use the columns in data_df for analytics. Please provide a prediction of the total points Lewis Hamilton may score in his debut season with Ferrari in 2025 using XGBoost.

## Prepare data for driver point prediction

### Subtask:
Select relevant columns from `data_df` and prepare the data for predicting driver points per race.


**Reasoning**:
Select the relevant columns from data_df and store them in a new DataFrame for driver points prediction.



In [122]:
driver_points_df = data_df[['year', 'raceId', 'driverId', 'constructorId', 'grid', 'positionOrder', 'points']].copy()
display(driver_points_df.head())
driver_points_df.info()

,year,raceId,driverId,constructorId,grid,positionOrder,points
0,2008,18,1,1,1,1,10.0
1,2008,18,2,2,5,2,8.0
2,2008,18,3,3,7,3,6.0
3,2008,18,4,4,11,4,5.0
4,2008,18,5,1,3,5,4.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27078 entries, 0 to 27077
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           27078 non-null  int64  
 1   raceId         27078 non-null  int64  
 2   driverId       27078 non-null  int64  
 3   constructorId  27078 non-null  int64  
 4   grid           27078 non-null  int64  
 5   positionOrder  27078 non-null  int64  
 6   points         27078 non-null  float64
dtypes: float64(1), int64(6)
memory usage: 1.4 MB


## Feature engineering

### Subtask:
Create relevant features for the model, such as driver and constructor performance metrics, grid position, etc.


**Reasoning**:
Calculate historical performance metrics for drivers and constructors, merge constructor winning percentage, and calculate race counts, then display the updated DataFrame.



In [125]:
# Sort data to ensure correct cumulative calculations
driver_points_df = driver_points_df.sort_values(by=['driverId', 'year', 'raceId'])

# Calculate cumulative points and race count for each driver
driver_points_df['cumulative_driver_points'] = driver_points_df.groupby('driverId')['points'].cumsum()
driver_points_df['driver_race_count'] = driver_points_df.groupby('driverId').cumcount() + 1

# Calculate historical average points per race for each driver (excluding current race)
# Use the cumulative points up to the *previous* race and the race count up to the *previous* race
driver_points_df['avg_driver_points'] = (driver_points_df['cumulative_driver_points'] - driver_points_df['points']) / (driver_points_df['driver_race_count'] - 1)
driver_points_df['avg_driver_points'] = driver_points_df['avg_driver_points'].fillna(0) # Fill NaN for the first race

# Calculate cumulative grid position and race count for each driver
driver_points_df['cumulative_driver_grid'] = driver_points_df.groupby('driverId')['grid'].cumsum()

# Calculate historical average grid position for each driver (excluding current race)
driver_points_df['avg_driver_grid'] = (driver_points_df['cumulative_driver_grid'] - driver_points_df['grid']) / (driver_points_df['driver_race_count'] - 1)
driver_points_df['avg_driver_grid'] = driver_points_df['avg_driver_grid'].fillna(driver_points_df['grid']) # Fill NaN for the first race with current grid

# Sort data by constructor for constructor calculations
driver_points_df = driver_points_df.sort_values(by=['constructorId', 'year', 'raceId'])

# Calculate cumulative points and race count for each constructor
driver_points_df['cumulative_constructor_points'] = driver_points_df.groupby('constructorId')['points'].cumsum()
driver_points_df['constructor_race_count'] = driver_points_df.groupby('constructorId').cumcount() + 1

# Calculate historical average points per race for each constructor (excluding current race)
driver_points_df['avg_constructor_points'] = (driver_points_df['cumulative_constructor_points'] - driver_points_df['points']) / (driver_points_df['constructor_race_count'] - 1)
driver_points_df['avg_constructor_points'] = driver_points_df['avg_constructor_points'].fillna(0) # Fill NaN for the first race

# Calculate cumulative grid position and race count for each constructor (using the driver's grid as representative of constructor performance in that race)
driver_points_df['cumulative_constructor_grid'] = driver_points_df.groupby('constructorId')['grid'].cumsum()

# Calculate historical average grid position for each constructor (excluding current race)
driver_points_df['avg_constructor_grid'] = (driver_points_df['cumulative_constructor_grid'] - driver_points_df['grid']) / (driver_points_df['constructor_race_count'] - 1)
driver_points_df['avg_constructor_grid'] = driver_points_df['avg_constructor_grid'].fillna(driver_points_df['grid']) # Fill NaN for the first race with current grid

# Merge constructor winning percentage
# Ensure constructor_win_percentage_df is available and has 'constructorId' and 'win_percentage'
if 'constructor_win_percentage_df' in locals() and not constructor_win_percentage_df.empty:
    driver_points_df = pd.merge(driver_points_df, constructor_win_percentage_df[['constructorId', 'win_percentage']], on='constructorId', how='left')
    driver_points_df['win_percentage'] = driver_points_df['win_percentage'].fillna(0) # Fill NaN for constructors not in the win percentage data

# Create interaction features (example: interaction between driver points and constructor points)
driver_points_df['driver_constructor_points_interaction'] = driver_points_df['avg_driver_points'] * driver_points_df['avg_constructor_points']

# Display the head and info of the updated DataFrame
display(driver_points_df.head())
driver_points_df.info()

,year,raceId,driverId,constructorId,grid,positionOrder,points,cumulative_driver_points,driver_race_count,avg_driver_points,cumulative_driver_grid,avg_driver_grid,cumulative_constructor_points,constructor_race_count,avg_constructor_points,cumulative_constructor_grid,avg_constructor_grid,win_percentage,driver_constructor_points_interaction
0,1968,674,360,1,16,13,0.0,151.5,85,1.803571,705,8.202381,0.0,1,0.000000,16,16.000000,0.0,0.000000
1,1968,677,304,1,5,8,0.0,107.0,37,2.972222,264,7.194444,0.0,2,0.000000,21,16.000000,0.0,0.000000
2,1971,632,304,1,7,6,1.0,155.0,61,2.566667,427,7.000000,1.0,3,0.000000,28,10.500000,0.0,0.000000
3,1971,632,320,1,11,23,0.0,1.0,8,0.142857,118,15.285714,1.0,4,0.333333,39,9.333333,0.0,0.047619
4,1971,632,347,1,23,24,0.0,39.0,105,0.375000,1199,11.307692,1.0,5,0.250000,62,9.750000,0.0,0.093750


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27078 entries, 0 to 27077
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   year                                   27078 non-null  int64  
 1   raceId                                 27078 non-null  int64  
 2   driverId                               27078 non-null  int64  
 3   constructorId                          27078 non-null  int64  
 4   grid                                   27078 non-null  int64  
 5   positionOrder                          27078 non-null  int64  
 6   points                                 27078 non-null  float64
 7   cumulative_driver_points               27078 non-null  float64
 8   driver_race_count                      27078 non-null  int64  
 9   avg_driver_points                      27078 non-null  float64
 10  cumulative_driver_grid                 27078 non-null  int64  
 11  av

## Data splitting

### Subtask:
Split the data into training and testing sets.


**Reasoning**:
Define the feature matrix X and the target vector y, then split the data into training and testing sets.



In [126]:
# Define feature matrix X (excluding target and identifying columns)
features = ['grid', 'avg_driver_points', 'avg_driver_grid',
            'avg_constructor_points', 'avg_constructor_grid', 'driver_race_count',
            'constructor_race_count', 'win_percentage', 'driver_constructor_points_interaction']

X = driver_points_df[features]

# Define target vector y
y = driver_points_df['points']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes to verify the split
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (21662, 9)
Shape of X_test: (5416, 9)
Shape of y_train: (21662,)
Shape of y_test: (5416,)


## Model training

### Subtask:
Train an XGBoost Regressor model to predict points per race.


**Reasoning**:
Train an XGBoost Regressor model to predict points per race using the training data.



In [127]:
from xgboost import XGBRegressor

# Instantiate an XGBRegressor model
xgb_reg = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Train the model
xgb_reg.fit(X_train, y_train)

print("XGBoost Regressor model trained successfully.")

XGBoost Regressor model trained successfully.


**Reasoning**:
Filter the data for 2025, identify Lewis Hamilton and Ferrari, prepare input data for prediction for each 2025 race, predict points using the trained model, and store the predictions.



**Reasoning**:
The previous command failed because the columns in `prediction_inputs_df` did not match the columns in `X_train`. This is likely due to missing calculations for the historical averages and counts for Lewis Hamilton and Ferrari, or an issue with how the `constructor_win_percentage_df` was created or merged. Need to ensure all required features are calculated and present in `prediction_inputs_df` before attempting to align columns with `X_train`. Also, the `constructor_win_percentage_df` needs to be correctly calculated and merged or accessed.



**Reasoning**:
The error persists, indicating that the columns required for prediction are still not correctly populated in the `prediction_inputs_df` DataFrame before attempting to align with `X_train.columns`. This suggests an issue in the calculation or assignment of the feature values within the loop. The most likely cause is that the historical performance calculations are resulting in NaN values for some features, or there's an issue with how the interaction feature is being calculated or added. I need to carefully review the feature creation logic within the loop and ensure that all required features are being added to the `input_row` dictionary with valid numerical values for every race in 2025. I will add print statements to inspect `input_row` before it's appended to `prediction_inputs_2025` and also check the columns and dtypes of `prediction_inputs_df` after it's created.



In [129]:
# First, check what years you actually have
print("Available years in dataset:")
available_years = sorted(data_df['year'].unique())
print(available_years)

# Filter for years 2006 onwards that you have
data_2006_onwards = data_df[data_df['year'] >= 2006].copy()
print(f"Data from 2006 onwards: {len(data_2006_onwards)} rows")
print(f"Years available: {sorted(data_2006_onwards['year'].unique())}")

# If you want to include 2025 for predictions, create the race structure
if 2025 not in available_years:
    # Get 2024 race structure to create 2025 template
    latest_year = data_2006_onwards['year'].max()
    races_template = data_2006_onwards[data_2006_onwards['year'] == latest_year][['name_x', 'raceId']].drop_duplicates()

    print(f"\nCreating 2025 race structure based on {latest_year} ({len(races_template)} races)")

    # Create 2025 race entries (without results, for prediction purposes)
    races_2025 = []
    for _, race in races_template.iterrows():
        races_2025.append({
            'year': 2025,
            'raceId': race['raceId'] + 1000,  # Offset to avoid ID conflicts
            'name_x': race['name_x'],
            # Add minimal required columns
            'points': 0,  # Will be predicted
            'grid': 5,    # Assumed average grid position
        })

    # Convert to DataFrame
    data_2025_synthetic = pd.DataFrame(races_2025)
    print(f"Created {len(data_2025_synthetic)} synthetic 2025 race entries")

Available years in dataset:
[np.int64(1950), np.int64(1951), np.int64(1952), np.int64(1953), np.int64(1954), np.int64(1955), np.int64(1956), np.int64(1957), np.int64(1958), np.int64(1959), np.int64(1960), np.int64(1961), np.int64(1962), np.int64(1963), np.int64(1964), np.int64(1965), np.int64(1966), np.int64(1967), np.int64(1968), np.int64(1969), np.int64(1970), np.int64(1971), np.int64(1972), np.int64(1973), np.int64(1974), np.int64(1975), np.int64(1976), np.int64(1977), np.int64(1978), np.int64(1979), np.int64(1980), np.int64(1981), np.int64(1982), np.int64(1983), np.int64(1984), np.int64(1985), np.int64(1986), np.int64(1987), np.int64(1988), np.int64(1989), np.int64(1990), np.int64(1991), np.int64(1992), np.int64(1993), np.int64(1994), np.int64(1995), np.int64(1996), np.int64(1997), np.int64(1998), np.int64(1999), np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(20

In [130]:
# First, check what years you actually have
print("Available years in dataset:")
available_years = sorted(data_df['year'].unique())
print(available_years)

# Filter for years 2006 onwards that you have
data_2006_onwards = data_df[data_df['year'] >= 2006].copy()
print(f"Data from 2006 onwards: {len(data_2006_onwards)} rows")
print(f"Years available: {sorted(data_2006_onwards['year'].unique())}")

# Create 2025 synthetic data if it doesn't exist
if 2025 not in available_years:
    # Get 2024 race structure to create 2025 template
    latest_year = data_2006_onwards['year'].max()
    races_template = data_2006_onwards[data_2006_onwards['year'] == latest_year][['name_x', 'raceId']].drop_duplicates()

    print(f"\nCreating 2025 race structure based on {latest_year} ({len(races_template)} races)")

    # Create 2025 race entries (without results, for prediction purposes)
    races_2025 = []
    for _, race in races_template.iterrows():  # Fixed the syntax error here
        races_2025.append({
            'year': 2025,
            'raceId': race['raceId'] + 1000,  # Offset to avoid ID conflicts
            'name_x': race['name_x'],
            # Add minimal required columns
            'points': 0,  # Will be predicted
            'grid': 5,    # Assumed average grid position
        })

    # Convert to DataFrame and add to main dataset
    data_2025_synthetic = pd.DataFrame(races_2025)
    print(f"Created {len(data_2025_synthetic)} synthetic 2025 race entries")

    # Add the synthetic 2025 data to your main dataset
    data_df = pd.concat([data_df, data_2025_synthetic], ignore_index=True)
    print(f"Added synthetic 2025 data to main dataset")

# NOW filter for 2025 data (this will now have data)
data_2025_df = data_df[data_df['year'] == 2025].copy()
print(f"\nFiltered 2025 data: {len(data_2025_df)} races")

# 2. Identify driverId for Lewis Hamilton and constructorId for Ferrari
lewis_hamilton_id = drivers_df[drivers_df['driverRef'] == 'hamilton']['driverId'].iloc[0]
ferrari_id = constructors_df[constructors_df['constructorRef'] == 'ferrari']['constructorId'].iloc[0]

print(f"Lewis Hamilton ID: {lewis_hamilton_id}")
print(f"Ferrari ID: {ferrari_id}")

# Recalculate constructor_win_percentage_df to ensure it's correct and includes Ferrari
constructor_championship_df_recalc = data_df.groupby(['year', 'raceId', 'constructorId'])['points'].sum().reset_index()
constructor_championship_df_recalc = constructor_championship_df_recalc.sort_values(by=['constructorId', 'year', 'raceId'])
constructor_championship_df_recalc['cumulative_points'] = constructor_championship_df_recalc.groupby('constructorId')['points'].cumsum()

last_race_per_year_recalc = constructor_championship_df_recalc.groupby('year')['raceId'].max().reset_index()
championship_data_recalc = constructor_championship_df_recalc.merge(last_race_per_year_recalc, on=['year', 'raceId'], how='inner')

championship_winners_recalc = championship_data_recalc.loc[championship_data_recalc.groupby('year')['cumulative_points'].idxmax()].copy()
championship_winners_recalc['won_championship'] = 1

championship_data_recalc = championship_data_recalc.merge(championship_winners_recalc[['year', 'constructorId', 'won_championship']],
                                              on=['year', 'constructorId'], how='left')
championship_data_recalc['won_championship'] = championship_data_recalc['won_championship'].fillna(0)

# Calculate win percentage based on the recalculated championship data
constructor_win_counts = championship_data_recalc[championship_data_recalc['won_championship'] == 1]['constructorId'].value_counts().reset_index()
constructor_win_counts.columns = ['constructorId', 'championship_wins']

total_years_in_championship_data = championship_data_recalc['year'].nunique()

# Get all unique constructors from constructors_df to include those who never won
all_constructors = constructors_df[['constructorId', 'name']].copy()

constructor_win_percentage_df = all_constructors.merge(constructor_win_counts, on='constructorId', how='left')
constructor_win_percentage_df['championship_wins'] = constructor_win_percentage_df['championship_wins'].fillna(0)
constructor_win_percentage_df['win_percentage'] = (constructor_win_percentage_df['championship_wins'] / total_years_in_championship_data) * 100

# Get the constructor win percentage for Ferrari from the newly calculated DataFrame
ferrari_win_percentage = constructor_win_percentage_df[
    constructor_win_percentage_df['constructorId'] == ferrari_id
]['win_percentage'].iloc[0] if ferrari_id in constructor_win_percentage_df['constructorId'].values else 0

print(f"Ferrari win percentage: {ferrari_win_percentage:.2f}%")

# 3. Prepare prediction input for each race in 2025
prediction_inputs_2025 = []
print(f"\nPreparing prediction inputs for {len(data_2025_df)} races...")

for index, race_2025 in data_2025_df.iterrows():
    # Calculate historical driver stats for Lewis Hamilton up to 2024 from driver_points_df
    hamilton_historical = driver_points_df[
        (driver_points_df['driverId'] == lewis_hamilton_id) &
        (driver_points_df['year'] <= 2024)
    ]
    avg_driver_points = hamilton_historical['points'].mean() if not hamilton_historical.empty else 0
    avg_driver_grid = hamilton_historical['grid'].mean() if not hamilton_historical.empty else 0
    driver_race_count = hamilton_historical.shape[0]

    # Calculate historical constructor stats for Ferrari up to 2024 from driver_points_df
    ferrari_historical = driver_points_df[
        (driver_points_df['constructorId'] == ferrari_id) &
        (driver_points_df['year'] <= 2024)
    ]
    avg_constructor_points = ferrari_historical['points'].mean() if not ferrari_historical.empty else 0
    avg_constructor_grid = ferrari_historical['grid'].mean() if not ferrari_historical.empty else 0
    constructor_race_count = ferrari_historical.shape[0]

    # Assume an average grid position for a top team in 2025
    assumed_grid = 5 # Example: assuming an average starting grid of 5 for a top team

    # Calculate interaction feature
    driver_constructor_points_interaction = avg_driver_points * avg_constructor_points

    # Create the input row as a dictionary
    input_row = {
        'grid': assumed_grid,
        'avg_driver_points': avg_driver_points,
        'avg_driver_grid': avg_driver_grid,
        'avg_constructor_points': avg_constructor_points,
        'avg_constructor_grid': avg_constructor_grid,
        'driver_race_count': driver_race_count,
        'constructor_race_count': constructor_race_count,
        'win_percentage': ferrari_win_percentage,
        'driver_constructor_points_interaction': driver_constructor_points_interaction
    }
    prediction_inputs_2025.append(input_row)

# Convert the list of dictionaries to a DataFrame
prediction_inputs_df = pd.DataFrame(prediction_inputs_2025)

# Debugging: Print columns of the prediction_inputs_df
print("Columns in prediction_inputs_df before aligning:")
print(prediction_inputs_df.columns.tolist())
print(f"Shape: {prediction_inputs_df.shape}")

print("\nColumns in X_train:")
print(X_train.columns.tolist())

print("\nFirst few rows of prediction_inputs_df:")
print(prediction_inputs_df.head())

# Ensure columns are in the same order as X_train
prediction_inputs_df = prediction_inputs_df.reindex(columns=X_train.columns, fill_value=0)

print(f"\nFinal prediction_inputs_df shape: {prediction_inputs_df.shape}")

# 4. Use the trained xgb_reg model to predict points
predicted_points_2025 = xgb_reg.predict(prediction_inputs_df)

# Ensure predictions are non-negative
predicted_points_2025 = np.maximum(0, predicted_points_2025)

# Store the predicted points along with race information
data_2025_df['predicted_points'] = predicted_points_2025

# 5. Display the predictions
print("\nPredicted points for Lewis Hamilton with Ferrari in 2025 races:")
display(data_2025_df[['year', 'name_x', 'predicted_points']])

# Calculate and print the total predicted points for the season
total_predicted_points_2025 = data_2025_df['predicted_points'].sum()
print(f"\nTotal predicted points for Lewis Hamilton with Ferrari in 2025: {total_predicted_points_2025:.2f}")

# Additional analysis
print(f"\nAverage points per race: {total_predicted_points_2025/len(data_2025_df):.2f}")
print(f"Number of races: {len(data_2025_df)}")

Available years in dataset:
[np.int64(1950), np.int64(1951), np.int64(1952), np.int64(1953), np.int64(1954), np.int64(1955), np.int64(1956), np.int64(1957), np.int64(1958), np.int64(1959), np.int64(1960), np.int64(1961), np.int64(1962), np.int64(1963), np.int64(1964), np.int64(1965), np.int64(1966), np.int64(1967), np.int64(1968), np.int64(1969), np.int64(1970), np.int64(1971), np.int64(1972), np.int64(1973), np.int64(1974), np.int64(1975), np.int64(1976), np.int64(1977), np.int64(1978), np.int64(1979), np.int64(1980), np.int64(1981), np.int64(1982), np.int64(1983), np.int64(1984), np.int64(1985), np.int64(1986), np.int64(1987), np.int64(1988), np.int64(1989), np.int64(1990), np.int64(1991), np.int64(1992), np.int64(1993), np.int64(1994), np.int64(1995), np.int64(1996), np.int64(1997), np.int64(1998), np.int64(1999), np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(20

,year,name_x,predicted_points
26759,2025,Australian Grand Prix,8.257371
26760,2025,Australian Grand Prix,8.257371
26761,2025,Australian Grand Prix,8.257371
26762,2025,Australian Grand Prix,8.257371
26763,2025,Australian Grand Prix,8.257371
...,...,...,...
27073,2025,Italian Grand Prix,8.257371
27074,2025,Italian Grand Prix,8.257371
27075,2025,Italian Grand Prix,8.257371
27076,2025,Italian Grand Prix,8.257371



Total predicted points for Lewis Hamilton with Ferrari in 2025: 2634.10

Average points per race: 8.26
Number of races: 319


**Reasoning**:
The debug output shows that `prediction_inputs_df` is an empty DataFrame (RangeIndex(start=0, stop=0, step=1)). This indicates that the loop iterating through `data_2025_df` is not executing because `data_2025_df` is empty. Looking back at the previous cells, `data_df` was loaded from `races.csv`, which contains race information, including 2025 races, but it doesn't contain result-specific information like points, grid positions, etc. The `consolidated_df` was created by merging race information with results, drivers, and constructors, but it seems `data_df` was later reassigned or filtered in a way that removed the 2025 races with the necessary result details or the merge process didn't correctly include 2025 races with result information.

The current subtask is to predict points for Lewis Hamilton in each race of the 2025 season using the trained XGBoost model. To do this, I need a DataFrame representing the 2025 races with columns that can be used to create the prediction input features (like 'grid', 'year', 'raceId', 'driverId', 'constructorId'). `data_df` is the designated main DataFrame, but it appears to be unsuitable in its current state for 2025 race prediction inputs based on the error.

Since the error is due to an empty 2025 DataFrame, I need to re-evaluate how to get the structure for 2025 races to create the prediction inputs. The `races_df` contains the 2025 race schedule. I can use this as a base and manually create the structure needed for prediction inputs for Lewis Hamilton and Ferrari for each race in 2025, incorporating the historical averages and the assumed grid position.

Therefore, I need to generate a DataFrame for the 2025 races with the required columns for prediction (`grid`, `avg_driver_points`, `avg_driver_grid`, `avg_constructor_points`, `avg_constructor_grid`, `driver_race_count`, `constructor_race_count`, `win_percentage`, `driver_constructor_points_interaction`) by iterating through the 2025 races found in `races_df` and populating the other feature columns with the pre-calculated historical values and assumptions.



In [131]:
# Identify driverId for Lewis Hamilton and constructorId for Ferrari
# Assuming drivers_df and constructors_df are available from previous steps
lewis_hamilton_id = drivers_df[drivers_df['driverRef'] == 'hamilton']['driverId'].iloc[0]
ferrari_id = constructors_df[constructors_df['constructorRef'] == 'ferrari']['constructorId'].iloc[0]

# Recalculate constructor_win_percentage_df to ensure it's correct and includes Ferrari
# This step is repeated from an earlier successful step to ensure the dataframe is available
# and up-to-date with the latest data_df state.
constructor_championship_df_recalc = data_df.groupby(['year', 'raceId', 'constructorId'])['points'].sum().reset_index()
constructor_championship_df_recalc = constructor_championship_df_recalc.sort_values(by=['constructorId', 'year', 'raceId'])
constructor_championship_df_recalc['cumulative_points'] = constructor_championship_df_recalc.groupby('constructorId')['points'].cumsum()

last_race_per_year_recalc = constructor_championship_df_recalc.groupby('year')['raceId'].max().reset_index()
championship_data_recalc = constructor_championship_df_recalc.merge(last_race_per_year_recalc, on=['year', 'raceId'], how='inner')

championship_winners_recalc = championship_data_recalc.loc[championship_data_recalc.groupby('year')['cumulative_points'].idxmax()].copy()
championship_winners_recalc['won_championship'] = 1

championship_data_recalc = championship_data_recalc.merge(championship_winners_recalc[['year', 'constructorId', 'won_championship']],
                                              on=['year', 'constructorId'], how='left')
championship_data_recalc['won_championship'] = championship_data_recalc['won_championship'].fillna(0)

# Calculate win percentage based on the recalculated championship data
constructor_win_counts = championship_data_recalc[championship_data_recalc['won_championship'] == 1]['constructorId'].value_counts().reset_index()
constructor_win_counts.columns = ['constructorId', 'championship_wins']

total_years_in_championship_data = championship_data_recalc['year'].nunique()

# Get all unique constructors from constructors_df to include those who never won
all_constructors = constructors_df[['constructorId', 'name']].copy()

constructor_win_percentage_df = all_constructors.merge(constructor_win_counts, on='constructorId', how='left')
constructor_win_percentage_df['championship_wins'] = constructor_win_percentage_df['championship_wins'].fillna(0)
constructor_win_percentage_df['win_percentage'] = (constructor_win_percentage_df['championship_wins'] / total_years_in_championship_data) * 100


# Get the constructor win percentage for Ferrari from the newly calculated DataFrame
ferrari_win_percentage = constructor_win_percentage_df[
    constructor_win_percentage_df['constructorId'] == ferrari_id
]['win_percentage'].iloc[0] if ferrari_id in constructor_win_percentage_df['constructorId'].values else 0


# Filter races_df for the year 2025 to get the race schedule
races_2025_df = races_df[races_df['year'] == 2025].copy()

# Prepare prediction input for each race in 2025
prediction_inputs_2025 = []
for index, race_2025 in races_2025_df.iterrows():
    # Calculate historical driver stats for Lewis Hamilton up to 2024 from driver_points_df
    hamilton_historical = driver_points_df[
        (driver_points_df['driverId'] == lewis_hamilton_id) &
        (driver_points_df['year'] <= 2024)
    ]
    avg_driver_points = hamilton_historical['points'].mean() if not hamilton_historical.empty else 0
    avg_driver_grid = hamilton_historical['grid'].mean() if not hamilton_historical.empty else 0
    driver_race_count = hamilton_historical.shape[0]

    # Calculate historical constructor stats for Ferrari up to 2024 from driver_points_df
    ferrari_historical = driver_points_df[
        (driver_points_df['constructorId'] == ferrari_id) &
        (driver_points_df['year'] <= 2024)
    ]
    avg_constructor_points = ferrari_historical['points'].mean() if not ferrari_historical.empty else 0
    avg_constructor_grid = ferrari_historical['grid'].mean() if not ferrari_historical.empty else 0
    constructor_race_count = ferrari_historical.shape[0]

    # Assume an average grid position for a top team in 2025
    # This is a simplification; a more complex model might predict grid position separately
    assumed_grid = 5 # Example: assuming an average starting grid of 5 for a top team

    # Calculate interaction feature
    driver_constructor_points_interaction = avg_driver_points * avg_constructor_points

    # Create the input row as a dictionary, ensuring all required features are present
    input_row = {
        'grid': assumed_grid,
        'avg_driver_points': avg_driver_points,
        'avg_driver_grid': avg_driver_grid,
        'avg_constructor_points': avg_constructor_points,
        'avg_constructor_grid': avg_constructor_grid,
        'driver_race_count': driver_race_count,
        'constructor_race_count': constructor_race_count,
        'win_percentage': ferrari_win_percentage, # Use Ferrari's calculated win percentage
        'driver_constructor_points_interaction': driver_constructor_points_interaction
    }
    prediction_inputs_2025.append(input_row)

# Convert the list of dictionaries to a DataFrame
prediction_inputs_df = pd.DataFrame(prediction_inputs_2025)

# Debugging: Print columns and info of the prediction_inputs_df
print("Columns in prediction_inputs_df before aligning:")
print(prediction_inputs_df.columns)
print("\nInfo of prediction_inputs_df before aligning:")
prediction_inputs_df.info()
print("\nColumns in X_train:")
print(X_train.columns)


# Ensure columns are in the same order as X_train
# Use reindex to handle potential missing columns in prediction_inputs_df (though ideally they should all be present)
# Check if prediction_inputs_df is empty before reindexing
if not prediction_inputs_df.empty:
    prediction_inputs_df = prediction_inputs_df.reindex(columns=X_train.columns, fill_value=0)
    # Debugging: Print info after reindexing
    print("\nInfo of prediction_inputs_df after reindexing:")
    prediction_inputs_df.info()
else:
    print("\nprediction_inputs_df is empty, cannot reindex.")


# 4. Use the trained xgb_reg model to predict points
# Check if prediction_inputs_df is empty before predicting
if not prediction_inputs_df.empty:
    predicted_points_2025 = xgb_reg.predict(prediction_inputs_df)

    # Ensure predictions are non-negative
    predicted_points_2025 = np.maximum(0, predicted_points_2025)

    # Store the predicted points along with race information from races_2025_df
    races_2025_df['predicted_points'] = predicted_points_2025

    # 5. Store the predicted points for each 2025 race (already done in the previous step)

    # Display the predictions
    print("\nPredicted points for Lewis Hamilton with Ferrari in 2025 races:")
    display(races_2025_df[['year', 'name', 'predicted_points']])

    # Calculate and print the total predicted points for the season
    total_predicted_points_2025 = races_2025_df['predicted_points'].sum()
    print(f"\nTotal predicted points for Lewis Hamilton with Ferrari in 2025: {total_predicted_points_2025:.2f}")
else:
    print("\nNo prediction could be made as the input DataFrame for 2025 races is empty.")


Columns in prediction_inputs_df before aligning:
Index(['grid', 'avg_driver_points', 'avg_driver_grid',
       'avg_constructor_points', 'avg_constructor_grid', 'driver_race_count',
       'constructor_race_count', 'win_percentage',
       'driver_constructor_points_interaction'],
      dtype='object')

Info of prediction_inputs_df before aligning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 9 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   grid                                   24 non-null     int64  
 1   avg_driver_points                      24 non-null     float64
 2   avg_driver_grid                        24 non-null     float64
 3   avg_constructor_points                 24 non-null     float64
 4   avg_constructor_grid                   24 non-null     float64
 5   driver_race_count                      24 non-null     int64  
 6 

,year,name,predicted_points
1125,2025,Australian Grand Prix,8.257371
1126,2025,Chinese Grand Prix,8.257371
1127,2025,Japanese Grand Prix,8.257371
1128,2025,Bahrain Grand Prix,8.257371
1129,2025,Saudi Arabian Grand Prix,8.257371
1130,2025,Miami Grand Prix,8.257371
1131,2025,Emilia Romagna Grand Prix,8.257371
1132,2025,Monaco Grand Prix,8.257371
1133,2025,Spanish Grand Prix,8.257371
1134,2025,Canadian Grand Prix,8.257371



Total predicted points for Lewis Hamilton with Ferrari in 2025: 198.18


## Estimate total points for 2025

### Subtask:
Sum the predicted points for all 2025 races to get an estimated total for Lewis Hamilton with Ferrari.


**Reasoning**:
Calculate the sum of the predicted points for the 2025 season and print the total.



In [132]:
total_predicted_points_2025 = races_2025_df['predicted_points'].sum()
print(f"Estimated total points for Lewis Hamilton with Ferrari in the 2025 season: {total_predicted_points_2025:.2f}")

Estimated total points for Lewis Hamilton with Ferrari in the 2025 season: 198.18
